## Holidays
https://public-apis.xyz/calendarific-8140 

In [184]:
import os 
import requests
import json 
import pandas as pd

from dotenv import load_dotenv, find_dotenv
assert load_dotenv(find_dotenv())

In [28]:
def write_holiday(country, year):
    url = 'https://calendarific.com/api/v2/holidays?&api_key={}&country={}&year={}'.format(os.environ["CALENDARIFIC_KEY"], country, year)
    headers = {'Accept': 'application/json'}
    #auth = ('username', 'userpass')
    response = requests.get(url, headers=headers)
    print("Writing with status: {}".format(response.status_code))
    
    with open('../data/external/holidays/holiday_{}{}.json'.format(country, year), 'w') as outf:
        outf.write(response.text)

In [26]:
country_codes = {
    'Finland': 'fi',
    'China': 'cn',
    'Sweden': 'se',
    'Russia': 'ru',
    'Germany': 'de',
    'UnitedKingdoms': 'gb'
}

years = ['2017', '2018', '2019']
countries = ['fi','cn', 'se', 'ru', 'de', 'gb']

for year in years: 
    for country in countries: 
        write_holiday(country, year)

Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200
Writing with status: 200


## Clean and write to csv 

In [175]:
types = pd.Series()

flatten = lambda l: [item for sublist in l for item in sublist]

for country in countries:
    for year in years:
        with open('../data/external/holidays/holiday_{}{}.json'.format(country, year)) as f:
            data = json.load(f)

            holidays = []
            for holiday in data['response']['holidays']:
                holidays.append([holiday['date']['iso'], holiday['type'][0], country])
            types = types.append(pd.Series(holidays))

In [176]:
df_holidays = pd.DataFrame.from_items(zip(types.index, types.values)).T
df_holidays = df_holidays.rename(columns={0: "Date", 1: "Type", 2: 'Country'})

/Users/henriksoderberg/opt/miniconda3/envs/junc/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  """Entry point for launching an IPython kernel.


In [126]:
# Indices for the holidays to be able to pivot 
num_holiday = dict()
holiday_num = dict()
for i, hol in enumerate(list(df_holidays['Type'].unique())):
    num_holiday[i] = hol
    holiday_num[hol] = i

print(num_holiday)
print(holiday_num)

{0: 'National holiday', 1: 'Observance', 2: 'Season', 3: 'Common local holiday', 4: 'Weekend', 5: 'Clock change/Daylight Saving Time', 6: 'Muslim', 7: 'Christian', 8: 'Local holiday', 9: 'Hebrew', 10: 'Orthodox', 11: 'Local observance', 12: 'Hinduism'}
{'National holiday': 0, 'Observance': 1, 'Season': 2, 'Common local holiday': 3, 'Weekend': 4, 'Clock change/Daylight Saving Time': 5, 'Muslim': 6, 'Christian': 7, 'Local holiday': 8, 'Hebrew': 9, 'Orthodox': 10, 'Local observance': 11, 'Hinduism': 12}


In [177]:
df_holidays['Type'] = df_holidays['Type'].replace(holiday_num)

In [178]:
date_types = pd.pivot_table(df_holidays, index='Date',values='Type',columns='Country', aggfunc='sum')

In [179]:
date_types = date_types.reset_index()
date_types = date_types.replace(num_holiday)

In [180]:
date_types

Country,Date,cn,de,fi,gb,ru,se
0,2017-01-01,National holiday,National holiday,National holiday,Hebrew,National holiday,National holiday
1,2017-01-02,Common local holiday,NaN,NaN,National holiday,National holiday,NaN
2,2017-01-03,NaN,NaN,NaN,Local holiday,National holiday,NaN
3,2017-01-04,NaN,NaN,NaN,NaN,National holiday,NaN
4,2017-01-05,NaN,NaN,NaN,NaN,National holiday,Observance
...,...,...,...,...,...,...,...
495,2019-12-24,NaN,Observance,Observance,Observance,NaN,Observance
496,2019-12-25,Observance,National holiday,National holiday,National holiday,NaN,National holiday
497,2019-12-26,NaN,National holiday,National holiday,National holiday,NaN,National holiday
498,2019-12-30,NaN,NaN,NaN,Hebrew,NaN,NaN


In [182]:
date_types.to_csv('../data/processed/holidays.csv')